## TASK
1. Produce a spreadsheet of all students with 2 columns:
    a. RA for each student.
    b. Whether the student has done an intentional interaction.

2. Produce 2nd spreadsheet comprising the ff:
    a. RA name
    b. Total no. of intentional interactions had.

## POSSIBLE SOLUTION:
1. Develop general algorithm to be used to identify RAs and intentional interaction status for all students.
    a. Iterate through list of all students
    b. For each student, determine if variations of first and lastname are present in Roster with high confidence.
    c. If name is present,
        - update intentional interaction status to YES
        - update RA column with name of RA from Roster.
    d. Make use of **Room Number** criteria to simplify entire process.
2.  a. For remaining students not captured by step 1, develop less precise algorithm that checks for presence of at least last name in Roster.
    b. Isolate all such students and filter manually.

In [ ]:
# Each RA in Gilbert Hall will likely follow the same format for all their students. So algorithm could be fine-tuned for each RA and they format they use to list student names.
# Limited no. of RAs makes work easier.

# Room number will always be present. (PS. Actually False)
# Check if either last name and room number are present, or first name and room number are present.

In [41]:
import pandas as pd

# importing spreadsheets
roster = pd.read_excel("../data/FYQ Roster- Spring.xlsx")
interactions = pd.read_excel("../data/Intentional Interactions  (Responses).xlsx")

## check successful import
print(roster.head())
print("\n", interactions.head())

# Extract only Gilbert Hall data
interactions_gil = interactions[interactions["Building "] == "Gilbert"]
roster_gil = roster[roster["BUILDING"] == "GILBE"]

# Compare total vs gilbert-only
print("All Responses: ", interactions.shape[0])
print("Gilbert Hall Responses", interactions_gil.shape[0])

print("All Students: ", roster.shape[0])
print("Gilbert Hall Students: ", roster_gil.shape[0])

# drop unnecessary columns from interactions data
interactions_gil = interactions_gil[["Your Name ", "Resident Names and Room Numbers"]]


# FIRST ROUND OF CHECKS (Firstname, Lastname, Room Number)
for student in roster_gil.index:
    for RA in interactions_gil.index:
        if roster_gil.at[student, "LAST"].lower() in interactions_gil.at[RA, "Resident Names and Room Numbers"].lower():
            if roster_gil.at[student, "FIRST"].lower() in interactions_gil.at[RA, "Resident Names and Room Numbers"].lower():
                if str(roster_gil.at[student, "ROOM"]).lower() in interactions_gil.at[RA, "Resident Names and Room Numbers"].lower():
                    roster_gil.at[student, "RA"] = interactions_gil.loc[RA, "Your Name "]
                    roster_gil.at[student, "INTENTIONAL INTERACTION"] = "YES"

# counter = 0
# for student in roster_gil.index:
#     if roster_gil.loc[student, "INTENTIONAL INTERACTION"] == "YES":
#         counter += 1
# print("count before:", counter)

# SECOND ROUND OF CHECKS (Firstname, Lastname)
# extract labels of rows where Intentional interaction is empty
no_interaction = []
for student in roster_gil.index:
    if str(roster_gil.loc[student, "INTENTIONAL INTERACTION"]) == "nan":
        no_interaction.append(student)
print(no_interaction)

# Use labels to iterate through the data frame and check only first name and last name; not room number.
for student in no_interaction:
    for RA in interactions_gil.index:
        if roster_gil.loc[student, "LAST"].lower() in interactions_gil.loc[RA,"Resident Names and Room Numbers"].lower():
            if roster_gil.loc[student, "FIRST"].lower() in interactions_gil.loc[RA, "Resident Names and Room Numbers"].lower():
                roster_gil.at[student, "INTENTIONAL INTERACTION"] = "YES"
                roster_gil.at[student, "RA"] = interactions_gil.loc[RA, "Your Name "]

# counter = 0
# for student in roster_gil.index:
#     if roster_gil.loc[student, "INTENTIONAL INTERACTION"] == "YES":
#         counter += 1
# print("count after:", counter)


roster_gil.to_excel("../output.xlsx")

All Responses:  906
Gilbert Hall Responses 386
All Students:  636
Gilbert Hall Students:  310
[11, 13, 32, 73, 74, 75, 98, 101, 121, 124, 141, 145, 149, 164, 165, 166, 173, 188, 201, 207, 208, 211, 241, 242, 247, 248, 249, 250, 262, 278, 281, 282, 284, 285, 286, 288, 289, 293, 299, 307, 309]


In [59]:
# Find number of Intentional interactions for each RA.

# Reimport data
roster_final = pd.read_excel("../data/COMPLETED FYQ Roster - Spring (March 23rd).xlsx")
interactions = roster_final["RA"]
histogram = {}
for interaction in interactions:
    histogram[interaction] = histogram.get(interaction, 0) + 1

col1 = histogram.keys()
col2 = histogram.values()
histogram_df = pd.DataFrame({"RA": col1, "No. of Intentional Interactions":col2})

histogram_df.to_excel("../output_2.xlsx")
roster_final

,LAST,FIRST,PREFERRED,BUILDING,ROOM,INTENTIONAL INTERACTION,RA,Unnamed: 7,Legend,Unnamed: 9
0,LOVELACE,AIDEN,AIDEN,GILBE,145,YES,Natan Sahilu,NaN,Color,Meaning
1,PARKER,DARIAN,DARIAN,GILBE,145,YES,Natan Sahilu,NaN,NaN,Actual student whose name is not in the intent...
2,MELTZER,JOSEPH,JOEY,GILBE,146,YES,Natan Sahilu,NaN,NaN,RA whose name is in the FYQ Roster list of stu...
3,GAE,RYAN,RYAN,GILBE,146,YES,Natan Sahilu,NaN,NaN,Miscellaneous problems that require case by ca...
4,DEB,ARINDAM,ARINDAM,GILBE,146,YES,Natan Sahilu,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
305,ZAHID,SHARMEEN,SHARMEEN,GILBE,491,YES,Karla,NaN,NaN,NaN
306,NGUYEN,ANH,BRIDGIT,GILBE,491,YES,Karla,NaN,NaN,NaN
307,LI,YINUO,YINUO,GILBE,492,NaN,NaN,NaN,NaN,NaN
308,ROJAS BOWE,GABRIELA,GABRIELA,GILBE,493,YES,Karla,NaN,NaN,NaN
